## Static Bayesian Networks

This notebook allows to fit static BN for the specified sector on the whole available period.

This notebook is made of the following main steps:

* Fit bayesian network using the library pgmpy. Structure learning is done on multiple boostrapped datasets and multi-core is exploited to parallelize the computations

* Perform parameter learning and make inference to determine links' strengths. Link strength is defined as P[ sink=1 | source=1 ] (probability of a jump in the sink node given a jump in the source node)

* Estimate partial correlations between linked issuers via tigramite

* Plot the resulting graphs

In [1]:
import pandas as pd
import numpy as np
import os
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator, TreeSearch, MmhcEstimator
import networkx as nx
from matplotlib import pyplot as plt
import time
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
from scipy.optimize import minimize_scalar
from statsmodels.distributions.empirical_distribution import ECDF
from pgmpy.models import BayesianNetwork
import time
from joblib import Parallel, delayed
from pgmpy.inference import VariableElimination
import sys
sys.path.append('../Modules')
from tigramite_utils import *
from utils_bayesian_networks import *

ModuleNotFoundError: No module named 'pgmpy'

In [ ]:
# Specify sector and status (period)
sector = 'Materials'
# status = 'm1'

# Define discrete status for the specified sector in the 3 periods and on the whole time horizon
define_discrete_states_based_on_volatility_jumps('Materials', 5, ['2019-01-01', '2020-10-01'],
                                                 '../Data/discrete_status_bn/')

In [ ]:
# Read discrete data for the specified status (period)

input_data = pd.read_csv('../Data/discrete_status_bn/discrete_data_' + sector + '_complete' + '.csv', sep=',').\
    set_index('Date', drop=True)

for i in input_data:
    input_data[i] = input_data[i].astype('category')

input_data

In [ ]:
input_data.shape

### BAYESIAN NETWORK LEARNING


1. Create all bootstrapped datasets


2. Create a df with columns from-to-strength-direction with all possible pairs of directed arcs


3. For each bootstrapped dataset:
   * Run HillClimbSearch.estimate to estimate the model
   * Update the number of times that each arc is estimated (taking direction into account) (columns "strength" and "direction" in the df from point 2)
   
  
4. Define "freq_direction" as frequency of each direction per each arc (e.g., A->B estimated 10 times, B->A estimated 4 times, then freq_direction(A->B) = 10/14 and freq_direction(B->A) = 4/14)


5. Define "freq_strength" as the number of times an arc (undirected) has been detected divided by the total number of experiments


6. Model Averaging:
   * Compute threshold for freq_strength (function used in the R library bnlearn)
   * Apply filter on arcs using the threshold
   
   
7. Parameter Learning


8. Inference to determine link strength

In [ ]:
# STEP 1: Bootstrapped datasets creation

seed = 0
num_bootstrap_dataset = 2000
bootstrap_datasets = []
np.random.seed(seed)

for i in range(num_bootstrap_dataset):
    bootstrap_datasets.append(input_data.sample(frac=1, replace=True).reset_index(drop=True))

### In case you want to use a previously saved model, go to [this cell](#saved_model_cell)

In [ ]:
# # #### NO PARALLELISM

# results_bootstrap = pd.DataFrame(list(input_data.columns), columns=['from']).\
#     merge(pd.DataFrame(list(input_data.columns), columns=['to']), how='cross').\
#     loc[lambda x: x['from'] != x['to']]
# results_bootstrap[['strength', 'direction']] = [0, 0]

# for i in range(len(bootstrap_datasets)):
#     df = bootstrap_datasets[i]
#     es = HillClimbSearch(df)
#     best_model = es.estimate(scoring_method=BicScore(df))

#     curr_results = pd.DataFrame(list(best_model.edges()), columns=['from', 'to']).assign(directed=True)
#     curr_results = pd.concat([curr_results, curr_results[['to', 'from']].rename(columns={'from': 'to', 'to': 'from'}).\
#                               assign(directed=False)])

#     results_bootstrap = results_bootstrap.merge(curr_results, how='left').\
#         assign(strength = lambda x: np.where(~x['directed'].isnull(), x['strength']+1, x['strength']),
#                direction = lambda x: np.where((x['directed']) & (~x['directed'].isnull()), x['direction']+1, x['direction'])).\
#         drop(columns=['directed'])
    


In [ ]:
# STEP 2-3: run estimate of dag on each bootstrapped dataset and collect results

start = time.time()

num_cores = 4
obs_per_core = int(len(bootstrap_datasets)/num_cores)
boot_df_batch = []
for i in range(num_cores):
    boot_df_batch.append(bootstrap_datasets[(obs_per_core*i):(obs_per_core*(i+1))])

def estimate_model(bootstrap_datasets, input_data):
    results_bootstrap = pd.DataFrame(list(input_data.columns), columns=['from']).\
        merge(pd.DataFrame(list(input_data.columns), columns=['to']), how='cross').\
        loc[lambda x: x['from'] != x['to']]
    results_bootstrap[['strength', 'direction']] = [0, 0]
    
    for i in range(len(bootstrap_datasets)):
        df = bootstrap_datasets[i]
        es = HillClimbSearch(df)
        best_model = es.estimate(scoring_method=BicScore(df))

        curr_results = pd.DataFrame(list(best_model.edges()), columns=['from', 'to']).assign(directed=True)
        curr_results = pd.concat([curr_results, curr_results[['to', 'from']].rename(columns={'from': 'to', 'to': 'from'}).\
                                  assign(directed=False)])

        results_bootstrap = results_bootstrap.merge(curr_results, how='left').\
            assign(strength = lambda x: np.where(~x['directed'].isnull(), x['strength']+1, x['strength']),
                   direction = lambda x: np.where((x['directed']) & (~x['directed'].isnull()), x['direction']+1, x['direction'])).\
            drop(columns=['directed'])
    
    return results_bootstrap

results = Parallel(n_jobs=num_cores)(delayed(estimate_model)(list_df, input_data) for list_df in boot_df_batch)

# Merge results
results_bootstrap = results[0]
for i in range(1, len(results)):
    results_bootstrap = results_bootstrap.\
        merge(results[i], how='inner', on=['from', 'to']).\
        assign(strength=lambda x: x['strength_x'] + x['strength_y'],
               direction=lambda x: x['direction_x'] + x['direction_y']).\
        drop(columns=['strength_x', 'strength_y', 'direction_x', 'direction_y'])
    
print('Elapsed time: ' + str(time.time() - start))

In [ ]:
# STEP 4-5: compute frequencies of appearance of arcs and directions

results_bootstrap['freq_strength'] = results_bootstrap['strength']/num_bootstrap_dataset
results_bootstrap['freq_direction'] = results_bootstrap['direction']/results_bootstrap['strength']

In [ ]:
# STEP 6: model averaging

e = ECDF(results_bootstrap['freq_strength'])
e(results_bootstrap['freq_strength'].sort_values())

u = results_bootstrap['freq_strength'].sort_values()
    
norm = lambda x: sum(np.diff(np.unique([0, *u, 1])) * abs(e(np.unique([0, *u[u < 1]])) - x))

p0 = minimize_scalar(norm, method='bounded', bounds=(0,1)).x

threshold = np.quantile(results_bootstrap['freq_strength'], p0)

significant_arcs = results_bootstrap.\
    loc[lambda x: (x['freq_strength'] >= threshold) & (x['freq_direction'] > 0.5)]

model_graph = nx.Graph()
model_graph.add_nodes_from(list(input_data.columns))
model_graph = model_graph.to_directed()
model_graph.add_edges_from(list(significant_arcs[['from', 'to']].to_records(index=False)))

if not nx.is_directed_acyclic_graph(model_graph):
    print('Making graph acyclic')
    for edge in list(model_graph.edges()):
        source = edge[0]
        sink = edge[1]
        new_graph = model_graph.copy()
        new_graph.remove_edge(source, sink)
        if nx.is_directed_acyclic_graph(new_graph):
            break
    model_graph = new_graph.copy()

plt.rcParams['figure.figsize'] = (18,10)
pos = graphviz_layout(model_graph, prog="dot")
nx.draw(model_graph, pos, with_labels=True, node_size = 3000, node_color='yellow')
plt.show()

In [ ]:
nx.is_directed_acyclic_graph(model_graph)

In [ ]:
# Remove arcs which were removed to avoid cycles

new_arcs = pd.DataFrame(columns=significant_arcs.columns)
list_edges = list(model_graph.edges())
for i in range(len(significant_arcs)):
    source = significant_arcs.iloc[i, :]['from']
    sink = significant_arcs.iloc[i, :]['to']
    
    if (source, sink) in list_edges:
      new_arcs = pd.concat([new_arcs, pd.DataFrame(significant_arcs.iloc[i, :]).T], ignore_index=True)
    
significant_arcs = new_arcs.copy()

In [ ]:
# Save results on all bootstrapped datasets and .csv file with all arcs
results_bootstrap.to_csv('../Output/5.1._BN_Results/results_pgmpy_plus_tigramite_2000_simulations_BKP_MODELS/' + sector + '_complete_res_boot.csv', sep=';', index=False)
# pd.DataFrame(list(model_graph.edges), columns=['from', 'to']).\
#     to_csv('../Output/5.1._BN_Results/' + sector + '_' + status + '_arcs.csv', sep=';', index=False)

In [ ]:
# STEP 7: Parameter Learning

bn_model = BayesianNetwork(model_graph)
bn_model.fit(input_data, estimator=MaximumLikelihoodEstimator)

for cpd in bn_model.get_cpds():
    print(cpd)
    
# bn_model.cpds is a list giving the conditional probabilities for each node of the network
# bn_model.cpds allows to understand each element of the list which node refers to
# e.g., bn_model.cpds[11] refers to conditional probabilities of STOIL given its parents ENI and TOTAL
# EXAMPLE: P(STOIL=0.5 | ENI=1, TOTAL=0)
# bn_model.cpds[11].get_value(STOIL = 0.5, ENI=1, TOTAL=0)

In [ ]:
# STEP 8: Inference to determine link strength

model_infer = VariableElimination(bn_model)

link_strength = {}
for edge in bn_model.edges():
    q = model_infer.query(variables=[edge[1]], evidence={edge[0]: 1}, show_progress=False)
    link_strength[edge] = q.values[2]

In [ ]:
# Save estimated model as .bif file

bn_model.save('../Output/5.1._BN_Results/results_pgmpy_plus_tigramite_2000_simulations_BKP_MODELS/' + sector + '_complete_model.bif')

<a id='saved_model_cell'></a>

### Run from the below cell onwards if you want to load a saved model

In [ ]:
# # Use this cell if you want to read a model saved in previous executions

# from pgmpy.readwrite import BIFReader
# reader = BIFReader("../Output/5.1._BN_Results/results_pgmpy_plus_tigramite_2000_simulations_BKP_MODELS/" + sector + "_complete_model.bif")
# model_bn = reader.get_model()
# model_graph = nx.Graph()
# model_graph.add_nodes_from(list(input_data.columns))
# model_graph = model_graph.to_directed()
# model_graph.add_edges_from(list(model_bn.edges()))
# pos = graphviz_layout(model_graph, prog="dot")

# new_arcs = pd.DataFrame(columns=[0,1])
# list_edges = list(model_bn.edges())
# for i in range(len(list_edges)):
#     source = list_edges[i][0]
#     sink = list_edges[i][1]
#     new_arcs = pd.concat([new_arcs, pd.DataFrame([source, sink]).T], ignore_index=True)
    
# significant_arcs = new_arcs.copy()
# significant_arcs.columns = ['from', 'to']

# # Inference to determine link strength
# model_infer = VariableElimination(model_bn)

# link_strength = {}
# for edge in model_bn.edges():
#     q = model_infer.query(variables=[edge[1]], evidence={edge[0]: '1.0'}, show_progress=False)
#     link_strength[edge] = q.values[2]
    

## Tigramite to determine intensity of relations

In [ ]:
# Read input data to compute partial correlations

df_all, df_all_lookup, _, _, _ =\
    import_data_clustering_issuer_rf(issuer_curves_file_name='../Data/Sprd_5Y_Gesav_import_Sector_values.csv',
                                    fx_rf_file_name='../Data/Observations - UniverseFX.csv',
                                    local_rf_file_name='../Data/Observations - UniverseLOCAL.csv')

df_complete = pd.read_csv('../Data/conditional_volatility_issuers.csv', sep=';')
df_complete['Date'] = pd.to_datetime(df_complete['Date'])
df_complete = df_complete.set_index('Date', drop=True)

In [ ]:
# Select only curves from a specified sector
curves_sector = df_all[['CurveID', 'MS_Sector']].loc[lambda x: x['CurveID'].isin(df_complete.columns)].\
    drop_duplicates().\
    loc[lambda x: x['MS_Sector'] == sector]['CurveID'].unique()
print('Number of curves in sector ' + sector + ': ' + str(len(curves_sector)))

df_sector = df_complete.loc[:,curves_sector]
df_sector.index = pd.to_datetime(df_complete.index)
df_sector.shape

In [ ]:
# Estimate partial correlation via tigramite for the different periods

significant_arcs, dict_edge_labels, node_colors = \
    compute_partial_correlation_tigramite(df_sector, df_periods=None, status=None, model_graph=model_graph, max_lag=30)
    

In [ ]:
## Define and plot new graph with the structure discovered by the bayesian network algorithm and quantitative information
## determined by the conditional indipendence tests carried out via tigramite

plt.rcParams['figure.figsize'] = (18,10)

model_graph_final = nx.Graph()
model_graph_final.add_nodes_from(list(input_data.columns))
model_graph_final = model_graph.to_directed()
model_graph_final.add_weighted_edges_from(list(significant_arcs[['from', 'to', 'value']].to_records(index=False)))

weights = [model_graph_final[u][v]['weight'] for u,v in model_graph_final.edges()]

nx.draw(model_graph_final, pos, with_labels=True, node_size = 3000, width=4,
        edge_color=weights, edge_cmap=plt.cm.coolwarm, edge_vmin=-1, edge_vmax=1,
        node_color=node_colors, cmap=plt.cm.Reds, vmin=0, vmax=1)
nx.draw_networkx_edge_labels(model_graph_final, pos, edge_labels=dict_edge_labels, rotate=False,
                            label_pos=0.5)

edge_bar = plt.colorbar(plt.cm.ScalarMappable(cmap=plt.cm.coolwarm, norm=plt.Normalize(vmin=-1, vmax=1)),
                        fraction=0.05, shrink=0.5, orientation='vertical')
edge_bar.ax.set_title('Cross correlation')

node_bar = plt.colorbar(plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(0,1)),
             fraction=0.05, shrink=0.5, orientation='vertical')
node_bar.ax.set_title('Auto correlation')

plt.savefig('../Output/5.1._BN_Results/results_pgmpy_plus_tigramite_2000_simulations/' + sector + '_complete.png')
plt.show()

In [ ]:
# Plot networks with link strength to determine edges' thickness and color

nx.draw(model_graph_final, pos, with_labels=True, node_size = 3000, width=[10*i for i in list(link_strength.values())],
            node_color=node_colors, cmap=plt.cm.Reds, vmin=0, vmax=1)
nx.draw_networkx_edge_labels(model_graph_final, pos,
                             edge_labels={k: np.round(link_strength[k], 2) for k in link_strength},
                             rotate=False, label_pos=0.5)

node_bar = plt.colorbar(plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(0,1)),
             fraction=0.05, shrink=0.5, orientation='vertical')
node_bar.ax.set_title('Auto correlation \n (node color)')

plt.savefig('../Output/5.1._BN_Results/results_pgmpy_plus_tigramite_2000_simulations/' + sector + '_complete_link_strength.png')
plt.show()

In [ ]:
# Produce .csv file with info about edges
info_corr = pd.DataFrame(dict_edge_labels.items()).\
    rename(columns={0: 'edge', 1: 'lag'})
info_corr[['from', 'to']] = pd.DataFrame(info_corr['edge'].tolist(), index=info_corr.index)
info_strength = pd.DataFrame(link_strength.items()).\
    rename(columns={0: 'edge', 1: 'link_strength'})
info_strength[['from', 'to']] = pd.DataFrame(info_strength['edge'].tolist(), index=info_strength.index)
significant_arcs[['from', 'to', 'value']].merge(info_corr[['from', 'to', 'lag']]).\
    rename(columns={'value': 'cross_corr'}).\
    merge(info_strength[['from', 'to', 'link_strength']]).\
    assign(link_strength = lambda x: np.round(x['link_strength'], 3),
           cross_corr = lambda x: np.round(x['cross_corr'], 3)).\
    to_csv('../Output/5.1._BN_Results/results_pgmpy_plus_tigramite_2000_simulations/' + sector + '_complete_info_edges.csv', sep=';', index=False)

In [15]:
# Plot partial cross-correlation between two issuers for all possible lags

target_issuer = 'PEMEX'
source_issuer = 'ENI'
id_target = np.where(np.array(var_names) == target_issuer)[0][0]
id_source = np.where(np.array(var_names) == source_issuer)[0][0]

fig = plt.figure()
fig.set_size_inches(18,10)
plt.plot(range(1,max_lag+1),results['val_matrix'][id_source, id_target, 1:].squeeze(), '--o')
plt.xticks(range(1,max_lag+1))
plt.xlabel('Lag')
plt.ylabel('Partial cross-correlation')
plt.title('Partial cross-correlation between lagged ' + source_issuer + ' and ' + target_issuer)
plt.grid()
plt.savefig('ENI_PEMEX_cross_correlation.png', facecolor='white')

SyntaxError: invalid syntax (2323107824.py, line 2)